<a href="https://colab.research.google.com/github/vivek6311/Artificial-Intelligence-with-Python/blob/master/Kantar_Brand_AWC_Calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

def calculate_house_level_with_brand_map(
    file_path,
    brand_map_file,
    sheet_name='Sheet1',
    output_file=None
):
    # Load data and mapping
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    brand_map = pd.read_excel(brand_map_file)  # Must have columns: Brand, House

    # Clean column names for safety
    df.columns = df.columns.str.strip()
    brand_map.columns = brand_map.columns.str.strip()

    # Validate required columns
    if 'User Pool' not in df.columns:
        raise ValueError("Column 'User Pool' not found in the data.")
    if 'Weight' not in df.columns:
        raise ValueError("Column 'Weight' not found in the data.")
    if not {'Brand', 'House'}.issubset(brand_map.columns):
        raise ValueError("Mapping file must have columns: 'Brand' and 'House'")

    # Calc total pool value & validate
    c2_value = df['User Pool'].iloc[0]
    weight_sum = df['Weight'].sum()
    if weight_sum == 0:
        raise ValueError("Sum of 'Weight' column is zero, cannot normalize.")

    # Normalised respondent weight
    df['Resp Weight'] = df['Weight'] * c2_value / weight_sum

    # Build case-insensitive column map for matching
    df_col_map = {col.strip().lower(): col for col in df.columns}

    # Process houses
    houses = brand_map['House'].unique()
    for house in houses:
        # Get all brands for this house
        brands = brand_map.loc[brand_map['House'] == house, 'Brand'].tolist()

        # Match to actual df columns (case-insensitive, space-trimmed)
        brand_cols = [
            df_col_map[b.strip().lower()]
            for b in brands if b.strip().lower() in df_col_map
        ]

        if brand_cols:
            # Only doing AWC SUM & AWC WEIGHTED SUM   (row-level)
            df[f'Brand AWC Calib: {house}'] = df[brand_cols].sum(axis=1)
            df[f'Disp Brand AWC Weighted Calib: {house}'] = (
                df['Weight'] * df[f'Brand AWC Calib: {house}']
            )
        else:
            df[f'Brand AWC Calib: {house}'] = 0
            df[f'Disp Brand AWC Weighted Calib: {house}'] = 0

    # ---------- NEW PART: Create the aggregated summary ----------
    summary_data = []
    for house in houses:
        # Filters for AWC > 0
        mask = df[f'Brand AWC Calib: {house}'] > 0

        total_awc = df.loc[mask, f'Brand AWC Calib: {house}'].sum()
        total_weighted_awc = df.loc[mask, f'Disp Brand AWC Weighted Calib: {house}'].sum()
        total_weights = df.loc[mask, 'Weight'].sum()

        # Equivalent to your Excel formula
        avg_awc_house = total_weighted_awc / total_weights if total_weights > 0 else 0

        summary_data.append({
            "House": house,
            "House AWC (Sum>0)": total_awc,
            "Weighted House AWC (Sum>0)": total_weighted_awc,
            "Avg House AWC (Weighted)": avg_awc_house
        })

    summary_df = pd.DataFrame(summary_data)

    # Save both sheets in the same file
    if output_file is None:
        base, ext = os.path.splitext(file_path)
        output_file = f"{base}_houseoutput{ext}"

    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name="Row Level Data")
        summary_df.to_excel(writer, index=False, sheet_name="House Level Summary")

    print(f"Output saved to: {output_file}")


# Example usage
calculate_house_level_with_brand_map(
    "/content/UK_RCS_data25.xlsx",  # main data file
    '/content/UK_RCS_Brands.xlsx',  # mapping file: Brand, House
    sheet_name='Sheet1'
)


/tmp/ipython-input-3037940133.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Disp Brand AWC Weighted Calib: {house}'] = (
/tmp/ipython-input-3037940133.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Brand AWC Calib: {house}'] = df[brand_cols].sum(axis=1)
/tmp/ipython-input-3037940133.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To 

Output saved to: /content/UK_RCS_data25_houseoutput.xlsx


In [3]:
import pandas as pd
import os

def calculate_house_level_with_brand_map(
    file_path,
    brand_map_file,
    sheet_name='Sheet1',
    output_file=None
):
    # Load data and mapping
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    brand_map = pd.read_excel(brand_map_file)  # Must have columns: Brand, House

    # Clean column names for safety
    df.columns = df.columns.str.strip()
    brand_map.columns = brand_map.columns.str.strip()

    # Clean Brand and House names for consistency (new and important)
    brand_map['Brand'] = brand_map['Brand'].astype(str).str.strip()
    brand_map['House'] = brand_map['House'].astype(str).str.strip()

    # Also remove spaces in DataFrame columns for matching
    df_col_map = {col.strip().lower(): col for col in df.columns}

    # Validate required columns
    if 'User Pool' not in df.columns:
        raise ValueError("Column 'User Pool' not found in the data.")
    if 'Weight' not in df.columns:
        raise ValueError("Column 'Weight' not found in the data.")
    if not {'Brand', 'House'}.issubset(brand_map.columns):
        raise ValueError("Mapping file must have columns: 'Brand' and 'House'")

    # Calc total pool value & validate
    c2_value = df['User Pool'].iloc[0]
    weight_sum = df['Weight'].sum()
    if weight_sum == 0:
        raise ValueError("Sum of 'Weight' column is zero, cannot normalize.")

    # Normalised respondent weight
    df['Resp Weight'] = df['Weight'] * c2_value / weight_sum

    # NORMALIZE: Clean up house list to ensure uniqueness
    # (after stripping, get unique)
    houses = brand_map['House'].dropna().unique()

    for house in houses:
        clean_house = house.strip()
        # Get all brands for this house, also stripped
        brands = brand_map.loc[
            brand_map['House'].str.strip() == clean_house, 'Brand'
        ].astype(str).str.strip().tolist()

        # Match actual DataFrame columns (case-insensitive, stripped)
        brand_cols = [
            df_col_map[b.strip().lower()]
            for b in brands if b.strip().lower() in df_col_map
        ]

        col_awc = f'Brand AWC Calib: {clean_house}'
        col_awc_weighted = f'Disp Brand AWC Weighted Calib: {clean_house}'

        if brand_cols:
            df[col_awc] = df[brand_cols].sum(axis=1)
            df[col_awc_weighted] = df['Weight'] * df[col_awc]
        else:
            df[col_awc] = 0
            df[col_awc_weighted] = 0

    # ---------- NEW PART: Create the aggregated summary ----------
    summary_data = []
    for house in houses:
        clean_house = house.strip()
        col_awc = f'Brand AWC Calib: {clean_house}'
        col_awc_weighted = f'Disp Brand AWC Weighted Calib: {clean_house}'

        mask = df[col_awc] > 0

        total_awc = df.loc[mask, col_awc].sum()
        total_weighted_awc = df.loc[mask, col_awc_weighted].sum()
        total_weights = df.loc[mask, 'Weight'].sum()

        avg_awc_house = total_weighted_awc / total_weights if total_weights > 0 else 0

        summary_data.append({
            "House": clean_house,  # always stripped name here
            "House AWC (Sum>0)": total_awc,
            "Weighted House AWC (Sum>0)": total_weighted_awc,
            "Avg House AWC (Weighted)": avg_awc_house
        })

    summary_df = pd.DataFrame(summary_data)

    # Save both sheets in the same file
    if output_file is None:
        base, ext = os.path.splitext(file_path)
        output_file = f"{base}_house_output{ext}"

    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name="Row Level Data")
        summary_df.to_excel(writer, index=False, sheet_name="House Level Summary")

    print(f"Output saved to: {output_file}")


# Usage example:
calculate_house_level_with_brand_map(
     "/content/UK_RCS_data25.xlsx",  # main data file
     '/content/UK_RCS_Brands.xlsx',  # mapping file: Brand, House
     sheet_name='Sheet1'
 )


/tmp/ipython-input-2947193353.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_awc_weighted] = df['Weight'] * df[col_awc]
/tmp/ipython-input-2947193353.py:63: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_awc] = df[brand_cols].sum(axis=1)
/tmp/ipython-input-2947193353.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

Output saved to: /content/UK_RCS_data25_house_output.xlsx


In [9]:
import pandas as pd
import os

def calculate_house_level_with_brand_map(
    file_path,
    brand_map_file,
    sheet_name='Sheet1',
    output_file=None
):
    # Load data and mapping
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    brand_map = pd.read_excel(brand_map_file)  # Must have columns: Brand, House

    # Clean column names for safety
    df.columns = df.columns.str.strip()
    brand_map.columns = brand_map.columns.str.strip()

    # Clean Brand and House names for consistency (new and important)
    brand_map['Brand'] = brand_map['Brand'].astype(str).str.strip()
    brand_map['House'] = brand_map['House'].astype(str).str.strip()

    # Also remove spaces in DataFrame columns for matching
    df_col_map = {col.strip().lower(): col for col in df.columns}

    # Validate required columns
    if 'User Pool' not in df.columns:
        raise ValueError("Column 'User Pool' not found in the data.")
    if 'Weight' not in df.columns:
        raise ValueError("Column 'Weight' not found in the data.")
    if not {'Brand', 'House'}.issubset(brand_map.columns):
        raise ValueError("Mapping file must have columns: 'Brand' and 'House'")

    # Calc total pool value & validate
    c2_value = df['User Pool'].iloc[0]
    weight_sum = df['Weight'].sum()
    if weight_sum == 0:
        raise ValueError("Sum of 'Weight' column is zero, cannot normalize.")

    # Normalised respondent weight
    df['Resp Weight'] = df['Weight'] * c2_value / weight_sum

    # NORMALIZE: Clean up house list to ensure uniqueness
    houses = brand_map['House'].dropna().unique()

    # For diagnostics: store info about mapping, matching, calculation for each house
    diagnostics = []

    for house in houses:
        clean_house = house.strip()
        # Get all brands from mapping for this house, also stripped
        mapped_brands = brand_map.loc[
            brand_map['House'].str.strip() == clean_house, 'Brand'
        ].astype(str).str.strip().tolist()

        # Find which brands actually match columns in df
        matched_brands = [
            b for b in mapped_brands if b.strip().lower() in df_col_map
        ]
        matched_cols = [
            df_col_map[b.strip().lower()]
            for b in mapped_brands if b.strip().lower() in df_col_map
        ]
        unmatched_brands = [
            b for b in mapped_brands if b.strip().lower() not in df_col_map
        ]

        col_awc = f'Brand AWC Calib: {clean_house}'
        col_awc_weighted = f'Disp Brand AWC Weighted Calib: {clean_house}'

        if matched_cols:
            df[col_awc] = df[matched_cols].sum(axis=1)
            df[col_awc_weighted] = df['Weight'] * df[col_awc]
        else:
            df[col_awc] = 0
            df[col_awc_weighted] = 0

        # Save the calculations for diagnostics
        diagnostics.append({
            'House': clean_house,
            'Mapped Brands': ", ".join(mapped_brands),
            'Matched Brands In Data': ", ".join(matched_brands),
            'Unmatched Brands': ", ".join(unmatched_brands),
            'AWC Column': col_awc,
            'AWC Weighted Column': col_awc_weighted
        })

    # ---------- Create the aggregated summary ----------
    summary_data = []
    for house in houses:
        clean_house = house.strip()
        col_awc = f'Brand AWC Calib: {clean_house}'
        col_awc_weighted = f'Disp Brand AWC Weighted Calib: {clean_house}'

        # Changed condition here: != 0 includes both positive and negative nonzero values
        mask = df[col_awc] != 0

        total_awc = df.loc[mask, col_awc].sum()
        total_weighted_awc = df.loc[mask, col_awc_weighted].sum()
        total_weights = df.loc[mask, 'Weight'].sum()

        avg_awc_house = total_weighted_awc / total_weights if total_weights > 0 else 0

        summary_data.append({
            "House": clean_house,
            "House AWC (Sum!=0)": total_awc,
            "Weighted House AWC (Sum!=0)": total_weighted_awc,
            "Avg House AWC (Weighted)": avg_awc_house
        })

    summary_df = pd.DataFrame(summary_data)
    diagnostics_df = pd.DataFrame(diagnostics)

    # Save all sheets in the same file
    if output_file is None:
        base, ext = os.path.splitext(file_path)
        output_file = f"{base}_house_output{ext}"

    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name="Row Level Data")
        summary_df.to_excel(writer, index=False, sheet_name="House Level Summary")
        diagnostics_df.to_excel(writer, index=False, sheet_name="Calculation Diagnostics")

    print(f"Output saved to: {output_file}")

# Usage example:
calculate_house_level_with_brand_map(
    "/content/UK W2-23 Resp level_RCS_Final.xlsx",  # main data file
    '/content/UK_RCS_Brands.xlsx',  # mapping file: Brand, House
    sheet_name='Sheet1'
)

/tmp/ipython-input-2022227944.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_awc_weighted] = df['Weight'] * df[col_awc]
/tmp/ipython-input-2022227944.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_awc] = df[matched_cols].sum(axis=1)
/tmp/ipython-input-2022227944.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmente

Output saved to: /content/UK W2-23 Resp level_RCS_Final_house_output.xlsx
